#### 4. Cointegration

Integration은 시계열 변수가 통계 검증을 위해 안정적 시계열이 되기까지의 필요한 차분을 뜻한다. 필요한 차분의 횟수를 적분 차수(order of integration)이라 하는데, 시계열의 적분 차수가 일정 상수 $d$일 때, 시계열의 선형 결합의 적분 차수가 $d$보다 작을 때 시계열 사이에 Cointegration이 존재한다고 한다

**4.1 Spurious Regression**

시계열 데이터를 다루다 보면 흔히 하는 실수 중 하나는, 두 변수간 아무런 관계가 없는데도 불구하고 시간의 변화로 인해 유의한 상관관계가 나타나는 현상을 뜻한다. 그 예시로 여름철 아이스크림 판매량과 익사자수의 상관관계, 미국의 수출 인덱스와 호주의 남성 기대 수명간의 상관관계 등이 있다.

$$X_t = X_{t-1} + u_t, ~~~ u_t \sim N(0,\sigma_u^2)$$
$$Y_t = Y_{t-1} + v_t, ~~~ v_t \sim N(0,\sigma_v^2)$$

확률보행 과정을 따르는 시계열 또는 적분된 시계열의 수준을 분석하는 경우에는 두 시계열이 아무런 관계가 없음에도 불구하고 통계적으로 유의하다는 결론을 낼 확률이 상당히 높게 나타나는 문제가 발생한다

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import statsmodels.api as sm

In [ ]:
phi = 0.9 # coefficient
mu = 0
sigma = 1
T = 250

np.random.seed(1000)
x0 = np.random.normal(mu, sigma)

np.random.seed(42)
x1 = np.random.normal(mu, sigma)

np.random.seed(1000)
series1 = [x0]
for t in range(1, T):
    x_t = phi * series1[t-1] + np.random.normal(mu, sigma)
    series1.append(x_t)

np.random.seed(42)
series2 = [x1]
for t in range(1, T):
    x_t = phi * series2[t-1] + np.random.normal(mu, sigma)
    series2.append(x_t)

plt.plot(np.array(series1).cumsum(), label = 'Series 1')
plt.plot(np.array(series2).cumsum(), label = 'Series 2')
plt.xlabel("Time")
plt.ylabel("Value")
plt.title("AR(1) Process")
plt.legend()
plt.show()

In [ ]:
ar_model = sm.OLS(series1, sm.add_constant(series2))
results = ar_model.fit()

print(results.summary())

위 경우와 같이 단순 AR 모형임에도 불구하고 음의 상관관계를 보인다. 이 경우, 잘못된 관계로 해석할 수도 있는데 이것을 가성적 회귀 현상이라고 한다

**4.2 Cointegrated VAR Models**

k-dimensional $\mathrm{VAR}(p)$ 모형을 따르는 시계열 $\mathbf{x}_t$를 고려하자

$$\mathbf{x}_t = \mathbf{\mu}_t + \mathbf{Φ_1 x_{t-1}} + ⋯ + \mathbf{\Phi_p x_{t-p}} + \mathbf{a_{t}}$$

여기서 만약 모든 행렬식 $|\mathbf{Φ}(B)|$의 0 부분이 unit circle의 바깥 부분에 있다면, $\mathbf{x}_t$는 unit-root stationary하다. 여기서 unit-root stationary하다는 것은 $I(0)$프로세스를 따른다는 것을 의미한다.

편의를 위해, 간단하게 $\mathbf{x}_t$를 order 1를 따르는 intergrated process라고 가정하자. $\mathrm{VAR}(p)$의 Error Correction Model (ECM)은 다음과 같이 나타낼 수 있다

$$\mathbf{\Delta x}_t = \mathbf{\mu}_t + \mathbf{Π x_{t-1}} + \mathbf{Φ_1^* Δx_{t-1}} ⋯ + \mathbf{\Phi_{p-1}^* Δ x_{t-p+1}} + \mathbf{a_{t}}$$

여기서 $\mathbf{Π_1 x_{t-1}}$를 error correction term이라고 한다. 이는 cointegration을 익히는데 있어서 매우 중요한 역할을 한다. $\mathbf{\Phi_i}$는 ECM representation에 따라 다음과 같이 표현 가능하다

$$\mathbf{\Phi}_1 = \mathbf{I + \Pi + \Phi}_1^*$$
$$\mathbf{\Phi}_i = \mathbf{\Phi}_i^* + \mathbf{\Phi}_{i-1}^*$$

$\mathbf{x}_t$는 거의 $I(1)$이라는 가정 하에, 위 식의 $\mathbf{\Delta x}_t$는 $I(0)$프로세스이다. 만약 $\mathbf{x}_t$가 unit root를 가지고 있다면, $|\mathbf{\Phi}(1)| = 0$이다. 따라서 $\mathbf{\Pi} = - \mathbf{\Phi}(1)$는 특이행렬(singular matrix)이다. 결과적으로, ECM을 고려하는데 다음 세 가지 상황을 생각할 수 있다

1. Rank$(\mathbf{\Pi}) = 0$일 경우, $\mathbf{\Pi} = \mathbf{0}$이고 $\mathbf{x}_t$는 cointegrated하지 않다는 것과 같다. 이로 인해 ECM의 식은 다음과 같이 기약된다

$$\mathbf{\Delta x}_t = \mathbf{\mu}_t  + \mathbf{Φ_1^* Δx_{t-1}} ⋯ + \mathbf{\Phi_{p-1}^* Δ x_{t-p+1}} + \mathbf{a_{t}}$$

2. Rank$(\mathbf{\Pi}) = k$일 경우, $|\mathbf{\Pi}(1)| \neq 0$이고 $\mathbf{x}_t$는 unit root가 아니다. 이는 $\mathbf{x}_t$가 $I(0)$이란 것과 같다. 이 경우 ECM model은 정보를 가지고 있는 모형이라 보기 힘들다
3. 0 < Rank$(\mathbf{\Pi}) = m$ < $k$일 경우, $\mathbf{\Pi}$를 다음과 같이 쓸 수 있다

$$\mathbf{\Pi = \alpha \beta^\prime}$$

이 경우의 ECM model은 다음과 같이 풀어 쓸 수 있다

$$\mathbf{\Delta x}_t = \mathbf{\mu}_t + \mathbf{\alpha \beta^\prime x_{t-1}} + \mathbf{Φ_1^* Δx_{t-1}} ⋯ + \mathbf{\Phi_{p-1}^* Δ x_{t-p+1}} + \mathbf{a_{t}}$$

**4.3 Cointegration Test**

가설을 하나 고려해 보자

$$H_0 : \mathrm{Rank}(\mathbf{\Pi}) = m ~~~~~ \mathrm{versus} ~~~~~ H_a : \mathrm{Rank}(\mathbf{\Pi})>m$$

Johansen은 검정을 위해 우도 비율 통계량 (Likelihood Ratio statistic)을 제안하였다

$$\mathrm{LR_{tr}}(m) = -(T-p) \sum_{i=m+1}^k \ln (1 - \hat\lambda_i)$$

만약 $\mathrm{Rank}(\mathbf{\Pi}) = m$이면, $\hat\lambda_i$sms $i>m$일때 작아야 한다. 따라서 $\mathrm{LR_{tr}}(m)$은 작은 값이어야 한다. 이 검정을 trace cointegration test라고 한다. unit root이 존재하기 때문에 $\mathrm{LR_{tr}}(m)$의 점근 분포는 $\Chi^2$분포가 아니라 표준 브라운 운동의 함수이다. 따라서, $\mathrm{LR_{tr}}(m)$의 critical value는 시뮬레이션을 통해 얻어야 한다

Johansen은 또한 공적분 벡터의 수를 결정하기 위해 순차적인 절차를 고려한다. 구체적으로 관심있는 가설은 다음과 같다

$$H_0 : \mathrm{Rank}(\mathbf{\Pi}) = m ~~~~~ \mathrm{versus} ~~~~~ H_a : \mathrm{Rank}(\mathbf{\Pi}) = m+1$$

$\mathrm{LR}$ ratio의 검정 통계량은

$$\mathrm{LR_{max}}(m) = -(T-p)\ln (1-\hat\lambda_{m+1}$$

주의 : 검정 통계량의 critical value는 표준이 아니므로 시뮬레이션을 통해 평가해야 한다

#### example 7

대한민국의 대표 이동통신사업자 KT(030200)와 SKT(017670)의 2020년 1월 1일부터 2023년 6월 30일까지의 데이터를 import하여 두 회사의 주가가 공적분 관계에 있는지 확인하라. 어떤 특징을 보이는가?

In [ ]:
import datetime

start_date = datetime.datetime(2020, 1, 1)
end_date = datetime.datetime(2023, 6, 30)

kt = pdr.DataReader("030200", "naver", start_date, end_date).astype(int).Close
skt = pdr.DataReader("017670", "naver", start_date, end_date).astype(int).Close

In [ ]:
kt_ret = np.log(kt / kt.shift(1)).dropna()
skt_ret = np.log(skt / skt.shift(1)).dropna()

공적분 검정은 statsmodels.tsa.stattools의 coint 함수를 통해 검정 가능하다

In [ ]:
from statsmodels.tsa.stattools import coint

In [ ]:
score, p_value, _ = coint(kt_ret, skt_ret)
print('Correlation : %.4f' % kt_ret.corr(skt_ret))
print('Cointegration test p-value : %.4f' % p_value)
print('Cointegration test score : %.4f'  % score)

공적분 검정에서 귀무가설(두 자산은 공적분 관계에 있지 않다)을 기각하므로, 두 자산은 공적분 관계에 있다는 것을 알 수 있다. 이는 두 자산이 장기적으로 서로 유의한 관계에 있다는 것을 알 수 있다

이에 반해 상관계수는 0.4815 정도로 상관계수가 높은 편이 아닌데, 상관계수가 높지 않더라도 공적분 관계에 있을 수 있다는 것을 알 수 있다

#### Exericse 5

KOSPI, NIKKEI, S&P 혹은 NASDAQ 상장사 중 두 회사를 선정하여 2020년 1월 1일부터 2023년 6월 30일까지의 자산수익률 행렬을 import하라. 그리고 두 자산간 공적분 검정을 시행하라. 두 자산간 Correlation을 계산하라. 어떤 특징을 보이는가?